In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# System paths
root_dir = "../dataset/"
trainset_dir = os.path.join(root_dir, "train_v2")
testset_dir = os.path.join(root_dir, "test_v2")
train_rle = os.path.join(root_dir, "train_ship_segmentations_v2.csv")

df = pd.read_csv(train_rle)

In [3]:
def enet_weighing(df, c=1.02):
    """Computes class weights as described in the ENet paper:

        w_class = 1 / (ln(c + p_class)),

    where c is usually 1.02 and p_class is the propensity score of that
    class:

        propensity_score = freq_class / total_pixels.

    References: https://arxiv.org/abs/1606.02147

    Arguments:
        df (pandas.DataFrame): data frame
        c (int, optional): hyper-parameter which restricts
            the interval of values of the weights. Default: 1.02.

    """
    num_images = len(df["ImageId"].unique())
    rle_series = df["EncodedPixels"]    
    
    total = 768 * 768 * num_images
    ship_count = 0
    for rle in rle_series:
        if not pd.isna(rle):
            # Count pixels that belong to ships by summing the length elements in the encoding
            rle_split = rle.split()
            for value in rle_split[1::2]:
                ship_count += int(value)

    # Compute propensity score and then the weights for each class
    class_count=np.array([total - ship_count, ship_count])
    propensity_score = class_count / total
    class_weights = 1 / (np.log(c + propensity_score))

    return class_weights

In [4]:
enet_weighing(df)

array([ 1.42340848, 47.8291847 ])

In [5]:
def median_freq_balancing(df):
    """Computes class weights using median frequency balancing as described
    in https://arxiv.org/abs/1411.4734:

        w_class = median_freq / freq_class,

    where freq_class is the number of pixels of a given class divided by
    the total number of pixels in images where that class is present, and
    median_freq is the median of freq_class.

    Arguments:
        df (pandas.DataFrame): 

    """
    num_images = len(df["ImageId"].unique())
    rle_series = df["EncodedPixels"]
    total = 768 * 768 * num_images  
    ship_total = 0
    ship_count = 0
    for rle in rle_series:
        if not pd.isna(rle):
            ship_total += 768 * 768
            
            # Count pixels that belong to ships by summing the length elements in the encoding
            rle_split = rle.split()
            for value in rle_split[1::2]:
                ship_count += int(value)

    # Compute the frequency and its median
    class_count=np.array([total - ship_count, ship_count])
    total = np.array([total, ship_total])
    freq = class_count / total
    med = np.median(freq)

    return med / freq

In [6]:
median_freq_balancing(df)

array([  0.5013302 , 188.44108246])